In [1]:
import requests
import numpy as np
from bs4 import BeautifulSoup
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import json

In [2]:
base_url = 'https://carigold.com/forum/forums/official-news-current-issues.72/'

In [3]:
def get_href(url):
    while True:
        try:
            r = requests.get(url)
            break
        except:
            time.sleep(1.0)
    soup = BeautifulSoup(r.content, "lxml")
    a = soup.find_all('a')
    a = [a_.get('href') for a_ in a if a_.get('href')]
    max_page = max([int(a_.split('page-')[-1]) for a_ in a if '/forum/forums/' in a_ and '/page-' in a_])
    a = [a_ for a_ in a if a_.startswith('/forum/threads/')]
    filtered = []
    for a_ in a:
        splitted = a_.split('/')
        if len(splitted) == 5 and splitted[-1] == '':
            filtered.append(a_)
    return filtered, max_page

filtered, max_page = get_href(base_url)

In [ ]:
max_worker = 10

In [ ]:
for i in tqdm(range(2, max_page + 1, max_worker)):
    aranged = np.arange(i, i + max_worker)
    urls = [f'{base_url}page-{a}' for a in aranged]
    
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(get_href, url): url for url in urls}

        for future in as_completed(futures):
            filtered.extend(future.result()[0])

In [ ]:
topics = list(set(filtered))

In [ ]:
with open('current-issues-topics.json', 'w') as fopen:
    json.dump(topics, fopen)

In [4]:
with open('current-issues-topics.json') as fopen:
    topics = json.load(fopen)

In [5]:
len(topics)

164779

In [6]:
topics[0]

'/forum/threads/imam-britain-tolak-dakwaan-pengaruhi-serangan-di-ottawa.539809/'

In [7]:
import time

def get_href_topic(url, topic = None):
    while True:
        try:
            r = requests.get(url)
            break
        except:
            time.sleep(1.0)
            
    soup = BeautifulSoup(r.content, "lxml")
    a = soup.find_all('a')
    a = [a_.get('href') for a_ in a if a_.get('href')]
    if topic:
        try:
            max_page = max([int(a_.split('page-')[-1]) for a_ in a if t in a_ and '/page-' in a_])
        except:
            max_page = -1
    else:
        max_page = -1
    div = soup.find_all('div', {'class': 'bbWrapper'})
    return [str(d) for d in div], max_page


t = topics[0]
url = f'https://carigold.com{t}'
filtered, max_page = get_href_topic(url, t)

In [8]:
import os
import json

!mkdir current-issues

mkdir: cannot create directory ‘current-issues’: File exists


In [14]:
sorted_topics = sorted(topics)
max_worker = 20

for no in tqdm(range(len(sorted_topics))):
    t = sorted_topics[no]
    
    filename = os.path.join('current-issues', f'{no}.json')
    if os.path.exists(filename):
        continue
        
    url = f'https://carigold.com{t}'
    current_issues, max_page = get_href_topic(url, t)
    
    for i in tqdm(range(2, max_page + 1, max_worker)):
        aranged = np.arange(i, i + max_worker)
        urls = [f'{url}page-{a}' for a in aranged]

        with ThreadPoolExecutor(max_workers=max_worker) as executor:
            futures = {executor.submit(get_href_topic, url): url for url in urls}

            for future in as_completed(futures):
                current_issues.extend(future.result()[0])
    
    with open(filename, 'w') as fopen:
        json.dump(current_issues, fopen)

 99%|████████████████████████████████████████████████████████████████████████████████████▍| 163781/164779 [11:19:52<18:57,  1.14s/it]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [15]:
!du -hs current-issues

5.1G	current-issues


In [16]:
from glob import glob

files = glob('current-issues/*.json')
len(files)

164779

In [17]:
texts = []
for f in tqdm(files):
    with open(f) as fopen:
        data = json.load(fopen)
    for d in data:
        texts.append(BeautifulSoup(d, "lxml").text)

100%|██████████████████████████████████| 164779/164779 [20:34<00:00, 133.45it/s]


In [18]:
len(texts)

6769922

In [19]:
texts = list(set(texts))
len(texts)

3585225

In [ ]:
with open('current-issues.json', 'w') as fopen:
    json.dump(texts, fopen)

In [3]:
# !zip -r current-issues.zip current-issues

In [2]:
!ls -lh current-issues.zip

-rw-r--r-- 1 husein husein 695M Mei  19 22:24 current-issues.zip
